In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from konlpy.tag import Okt
import re
from collections import defaultdict
from tqdm import tqdm
import torch

# 데이터 불러오기
df1 = pd.read_excel("news/중앙일보(1).xlsx")

# GPU 설정
device = 0 if torch.cuda.is_available() else -1

# 모델 및 파이프라인 설정 (GPU 사용)
tokenizer = AutoTokenizer.from_pretrained("nlp04/korean_sentiment_analysis_kcelectra")
model = AutoModelForSequenceClassification.from_pretrained("nlp04/korean_sentiment_analysis_kcelectra")
sentiment_analyzer = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)

# 전처리 + 문장 분할 함수
def split_sentences(content):
    if not isinstance(content, str):
        return []
    text = re.sub(r'[^\.\?\!\s\w\d]', ' ', content)
    text = re.sub(r'([\.\?\!])', r'\1 ', text)
    sentences = re.split(r'[\.\?\!]\s+', text)
    return [s.strip() for s in sentences if s.strip()]

# 전체 문서를 문장 단위로 나누고 문서 ID 부여
all_sentences = []
doc_ids = []

for i, content in enumerate(tqdm(df1['본문'], desc="문장 분리 중")):
    sentences = split_sentences(content)
    all_sentences.extend(sentences)
    doc_ids.extend([i] * len(sentences))  # 문서 인덱스를 같이 저장

# 배치 감성 분석 (GPU에서 실행)
batch_size = 32
results = []
for i in tqdm(range(0, len(all_sentences), batch_size), desc="감성 분석 중"):
    batch = all_sentences[i:i+batch_size]
    try:
        result = sentiment_analyzer(batch)
        results.extend(result)
    except Exception as e:
        print(f"오류 발생: {e}")
        results.extend([{'label': 'error', 'score': 0.0}] * len(batch))  # 에러 발생 시 기본값

# 결과 집계: 문서별 평균 점수 계산
score_dict = defaultdict(lambda: defaultdict(list))

for doc_id, result in zip(doc_ids, results):
    label = result['label']
    score = result['score']
    score_dict[doc_id][label].append(score)

# 평균 점수 계산
final_scores = []
for i in range(len(df1)):
    label_scores = score_dict[i]
    avg_scores = [(label, round(sum(scores)/len(scores), 4)) for label, scores in label_scores.items()]
    final_scores.append(avg_scores)

# 결과 저장
df1['감성점수'] = final_scores


Device set to use cpu
감성 분석 중:   0%|                                                                           | 0/1633 [00:00<?, ?it/s]C:\Users\user\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `ElectraSelfAttention.forward`.
  return forward_call(*args, **kwargs)
감성 분석 중:   2%|█▌                                                                | 39/1633 [01:03<41:51,  1.58s/it]